<a href="https://prediktera.com" target="_blank">
    <img width="255" alt="Prediktera AB Breeze" src="https://prediktera.com/wp/wp-content/uploads/2022/02/prediktera_breeze_darkmode.png" style="background-color: black; border-radius: 25px;padding:10px;"/>
</a>
<br />
This is a sample tutorial for training your own YOLOv5 deep learning object detection network by Prediktera.

See <a href="https://github.com/Prediktera/Train-Onnx-Image-Segmentation/issues/new">GitHub</a> for community support or <a href="https://prediktera.com/contact-us">contact us</a> for professional support.

# Training YOLOv5 on Colab

Create Folder in 'My Drive', let's call it 'project_folder'. Create folder 'datasets' in the new project folder. Import the data into 'datasets' (for example, a folder named 'NutType' with export data from Breeze).

project_folder/datasets/BreezeExample/images
                                     /labels
                                     /coco.names
                                     /data.yaml

Install relevant dependencies, this may prompt you to restart the Runtime

In [ ]:
%pip install -q torch onnx onnxruntime onnxruntime-gpu onnx-simplifier --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
!git clone https://github.com/ultralytics/yolov5 --depth 1
%cd yolov5
%pip install -qr requirements.txt

In [ ]:
import torch
import utils
display = utils.notebook_init() 

Go to Runtime menu and click 'Restart runtime' or a buttom above `RESTART RUNTIME`

## For significantly faster training use the GPU type Runtime
First, you'll need to enable GPUs for the notebook:

*   Navigate to Edit→Notebook Settings
*   select GPU from the Hardware Accelerator drop-down

The training will still work without GPU but traning time will increase dramatically. 

Confirm that GPU is enabled with the code below.

In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name() != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('GPU Enabled')

## Perpare the data

Enter data name and name of yaml file in the code cell below.

In [ ]:
# data name and name of yaml file
data_name = 'NutType'
yaml_name = 'data.yaml'
project_folder = 'MyDrive/project_folder/datasets'
google_drive_mount = '/content/gdrive'
mounded_project_folder = google_drive_mount + '/' + project_folder
data_set_folder = mounded_project_folder + '/' + data_name
data_set_yaml_file = data_set_folder + '/data.yaml'
runs_folder = 'runs'

# Model information
img_size = 96
number_of_epochs = 300

In [ ]:
from google.colab import drive
drive.mount(google_drive_mount)

Fix paths in the YAML file, this is safe to run multiple times and will only update the paths if necessary

In [ ]:
# Fix location path in Yaml file
import yaml

with open(data_set_yaml_file, 'r+') as f:
    data_list = yaml.load(f, Loader=yaml.FullLoader)
    data_list['path'] = data_set_folder
    yaml.dump(data_list, f)

## Train the model

Train, export and validate the model using parameters set in the begining of the notebook

In [ ]:
import shutil

shutil.rmtree(runs_folder, ignore_errors=True, onerror=None)

exp_folder = runs_folder + '/train/exp'
exp_val_folder = runs_folder + '/val/exp'
exp_best_w = exp_folder + "/weights/best.pt"
exp_onnx = exp_folder + "/weights/best.onnx"

!python train.py --imgsz $img_size --batch-size -1 --epochs $number_of_epochs --data $data_set_yaml_file --weights yolov5s.pt --cache --freeze 10
!python export.py --imgsz $img_size --weights $exp_best_w --simplify --opset 13 --include onnx 
!python val.py --imgsz $img_size --data $data_set_yaml_file --weights $exp_onnx --task test

## Review training results

Display some data from the validation of the best fitting model created

In [ ]:
from IPython.display import Image
from IPython.core.display import display 
conf_png = exp_val_folder + "/confusion_matrix.png"
val_jpg = exp_val_folder + "/val_batch0_pred.jpg"
display(Image(filename =conf_png, height=500),Image(filename=val_jpg, height=500))

## Download complete onnx YOLOv5 model

Run the block below to download the created .onnx file or find it in your Google Drive.

In [ ]:
from google.colab import files
files.download(exp_onnx)